In [1]:
data_path = "/data-restricted/home/cheick_ba/data_on_nas/raphtory/data"

In [2]:
data_path = "/Users/cheick/Pometry/data/"

# Enron (loading)

In [3]:
import os
import re
import pandas as pd
import altair as alt
import raphtory as rp
from email.utils import parsedate_to_datetime, parsedate
#from datetime import timezone, datetime
from time import mktime

In [4]:
csv = pd.read_csv(os.path.join(data_path,'enrondata/archive.zip'), compression="zip")
emails = csv['message']

In [5]:
def extract_cc_list(text):
    cc_cut = text.split("Cc: ")
    
    if len(cc_cut) > 1:
        cc_section = cc_cut[1].split(":")[0]
        cc_emails = re.findall(r'\S+@\S+', cc_section)
        return list(map(lambda email: email.split('@')[0], cc_emails))
    else:
        return 

def extract_bcc_list(text):
    bcc_cut = text.split("Bcc: ")

    if len(bcc_cut) > 1:
        bcc_section = bcc_cut[1].split(":")[0]
        bcc_emails = re.findall(r'\S+@\S+', bcc_section)
        return list(map(lambda email: email.split('@')[0], bcc_emails))
    else:
        return

extract_sender = lambda text: text.split("From: ")[1].split("@")[0]
extract_recipient = lambda text: text.split("To: ")[1].split("@")[0]
extract_date = lambda text: text.split("Date: ")[1].split("\n")[0]

enron = pd.DataFrame()
enron['src'] = emails.apply(extract_sender)
enron['dst'] = emails.apply(extract_recipient)
enron['time'] = emails.apply(extract_date)
enron['type'] = 'direct'
enron['message'] = emails

enron_cc = enron.loc[:, ['src', 'time', 'message']]
enron_cc['dst'] = emails.apply(extract_cc_list)     
enron_cc = enron_cc.explode('dst').dropna()
enron_cc['type'] = 'cc'

enron_bcc = enron.loc[:, ['src', 'time', 'message']]
enron_bcc['dst'] = emails.apply(extract_bcc_list)     
enron_bcc = enron_bcc.explode('dst').dropna()
enron_bcc['type'] = 'bcc'

enron = pd.concat([enron, enron_cc, enron_bcc])
enron

,src,dst,time,type,message
0,phillip.allen,tim.belden,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",direct,Message-ID: <18782981.1075855378110.JavaMail.e...
1,phillip.allen,john.lavorato,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",direct,Message-ID: <15464986.1075855378456.JavaMail.e...
2,phillip.allen,leah.arsdall,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",direct,Message-ID: <24216240.1075855687451.JavaMail.e...
3,phillip.allen,randall.gay,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",direct,Message-ID: <13505866.1075863688222.JavaMail.e...
4,phillip.allen,greg.piper,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",direct,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...,...,...,...
517323,john.zufferli,kimberly.hillis,"Wed, 13 Jun 2001 07:19:27 -0700 (PDT)",bcc,Message-ID: <15205398.1075842036502.JavaMail.e...
517335,john.zufferli,rob.milnthorp,"Thu, 24 Jan 2002 14:00:42 -0800 (PST)",bcc,Message-ID: <20351161.1075842028492.JavaMail.e...
517352,john.zufferli,rob.milnthorp,"Fri, 18 Jan 2002 06:52:25 -0800 (PST)",bcc,Message-ID: <12099366.1075842028905.JavaMail.e...
517371,john.zufferli,jzufferli,"Mon, 7 Jan 2002 13:01:08 -0800 (PST)",bcc,Message-ID: <537849.1075842029343.JavaMail.eva...


In [6]:
g = rp.Graph()
for idx, record in enron.iterrows():
    g.add_edge(timestamp = record['time'], src=record['src'], dst=record['dst'], 
                       properties={'message': record['message']}, layer=record['type'])

print(g)
g.save_to_file("mygraph")

Graph(number_of_edges=159480, number_of_vertices=54072, earliest_time=315532800000, latest_time=2335560538000)


KeyboardInterrupt: 